# Library importing

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import joblib

# data pre-proccessing

## data reading

In [2]:
final_data = pd.read_csv("/kaggle/input/depi-dataset/data.csv")

In [3]:
categorical_cols = ['store_id', 'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2', 'state_id', 'item_category', 'item_subcategory']
numeric_cols=['wm_yr_wk', 'wday', 'snap', 'year', 'month', 'day', 'lag_1','price_flag','item_number']
for col in categorical_cols:
    if final_data[col].dtype == "object":
        final_data[col] = final_data[col].astype("category")
for col in numeric_cols:
    final_data[col] = pd.to_numeric(final_data[col], downcast="integer")
final_data['sell_price'] = pd.to_numeric(final_data['sell_price'], downcast="float")
del categorical_cols
del numeric_cols

In [4]:
final_data.drop(columns= 'item_id_enc',inplace = True)

## Data spliting

In [5]:
final_data.to_csv("/kaggle/working/data.csv",index= False)

In [6]:
preprocessor = joblib.load("/kaggle/input/depi-dataset/pre-proccess.pkl")

In [7]:
joblib.dump(preprocessor,"/kaggle/working/pre-proccess.pkl")

['/kaggle/working/pre-proccess.pkl']